This file runs each step of pipeline seperately as a debug tool

In [133]:
from tfx import v1 as tfx
from pipeline.config import pipe_config
# import tensorflow_decision_forests as tfdf

In [134]:
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

# Connect to Pipeline output

In [135]:
# from ml_metadata import metadata_store
# connection_config = metadata_store_pb2.ConnectionConfig()
# connection_config.sqlite.filename_uri = pipe_config.METADATA_PATH
# store = metadata_store.MetadataStore(connection_config,enable_upgrade_migration = True)
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    pipe_config.METADATA_PATH)

In [136]:
# get lattest trainer artifacts
from pipeline.schema_pipeline.utils import get_latest_artifacts
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

with Metadata(metadata_connection_config) as metadata_handler:
    stat_gen_output = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                         'StatisticsGen')
    stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

    ev_output = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                   'ExampleValidator')
    anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

    trainer_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'Trainer')
    example_gen_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'CsvExampleGen')
    evaluator_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'Evaluator')
    eval_artifact = evaluator_outputs[standard_component_specs.EVALUATION_KEY][0]
    

In [137]:
evaluator_outputs

defaultdict(list,
            {'evaluation': [Artifact(artifact: id: 36
              type_id: 26
              uri: "pipeline_output/penguin-e2e/Evaluator/evaluation/33"
              custom_properties {
                key: "name"
                value {
                  string_value: "penguin-e2e:2021-08-28T05:10:37.674821:Evaluator:evaluation:0"
                }
              }
              custom_properties {
                key: "tfx_version"
                value {
                  string_value: "1.2.0"
                }
              }
              state: LIVE
              create_time_since_epoch: 1630127477669
              last_update_time_since_epoch: 1630127477669
              , artifact_type: id: 26
              name: "ModelEvaluation"
              )],
             'blessing': [Artifact(artifact: id: 37
              type_id: 25
              uri: "pipeline_output/penguin-e2e/Evaluator/blessing/33"
              custom_properties {
                key: "baseline_m

## Stats Gen

In [138]:
visualize_artifacts(stats_artifacts)

## Example Validator

In [139]:

visualize_artifacts(anomalies_artifacts)

/Users/robertfu/Projects/tfx-e2e/penguin-pipeline/.venv/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Trainer

In [140]:
model_run_artifact_dir = trainer_outputs['model_run'][0].uri

In [141]:
# %load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

## Evaluator

In [142]:
import tensorflow_model_analysis as tfma

In [143]:
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py tensorflow_model_analysis
# !jupyter labextension install tensorflow_model_analysis@0.33.0

In [144]:
!jupyter nbextension list
!jupyter labextension list  # for JupyterLab


Known nbextensions:
  config dir: /Users/robertfu/.jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
      tensorflow_model_analysis/extension  enabled 
      - Validating: OK
  config dir: /Users/robertfu/Projects/tfx-e2e/penguin-pipeline/.venv/etc/jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
JupyterLab v3.1.9
/Users/robertfu/Projects/tfx-e2e/penguin-pipeline/.venv/share/jupyter/labextensions
        @jupyter-widgets/jupyterlab-manager v3.0.0 enabled OK (python, jupyterlab_widgets)

Other labextensions (built into JupyterLab)
   app dir: /Users/robertfu/Projects/tfx-e2e/penguin-pipeline/.venv/share/jupyter/lab
        tensorflow_model_analysis v0.33.0 enabled OK


Build recommended, please run `jupyter lab build`:
    @jupyterlab/docprovider-extension needs to be removed from build
    @jupyterlab/application changed from ~3.1.8 to ~3.0.0
    @jupyterlab/application-ext

In [164]:
eval_result = tfma.load_eval_result(eval_artifact.uri,model_name="candidate")

In [165]:
tfma.view.render_slicing_metrics(eval_result,slicing_column='sex')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'sex:male', 'metrics'…

In [148]:
tfma.addons.fairness.view.widget_view.render_fairness_indicator(eval_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'male', 'slice': 'sex:male', 'metrics': {'sparse_catego…

In [161]:
tfma.view.render_plot(eval_result, tfma.SlicingSpec(feature_values={"sex": "male"}))

PlotViewer(config={'sliceName': 'sex:male', 'metricKeys': {'calibrationPlot': {'metricName': 'calibrationHisto…

In [160]:
tfma.view.render_plot(eval_result, tfma.SlicingSpec()) # whole dataset

PlotViewer(config={'sliceName': 'Overall', 'metricKeys': {'calibrationPlot': {'metricName': 'calibrationHistog…